In [1]:
!pip install -q findspark
!pip install -q unidecode
!pip install -q pyspark
import findspark
findspark.init()

import requests
import json
import unidecode
from pyspark.sql import SparkSession

spark = SparkSession.builder \
      .master("local[1]") \
      .appName("Teste_Spark_Autoglass_candidatos") \
      .getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 20.3 MB/s eta 0:00:00


In [5]:
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [6]:
client_id = '{}'
client_secret = '{}'

In [7]:
url = 'https://accounts.spotify.com/api/token'

params = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret
}

response = requests.post(url, params).json()
access_token = response['access_token']

In [8]:
headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}

In [32]:
# variáveis adicionais
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### Consuma a API e extraia as informações de músicas e artistas dos seguintes genêros("Rock Nacional", "Piseiro/Arrocha" e "Pop Internacional";

In [20]:
generos = ["Rock Nacional", "Piseiro/Arrocha", "Pop Internacional"]
musicas = []

for genero in generos:
    resultados = sp.search(q=f'genre: "{genero}"', type='track,artist', limit=50)
    for track in resultados['tracks']['items']:
        musica = {
            'nome': track['name'],
            'artista': track['artists'][0]['name'],
            'genero': genero
        }
        musicas.append(musica)
    for artist in resultados['artists']['items']:
        artista = {
            'nome': artist['name'],
            'genero': genero
        }
        musicas.append(artista)

rdd = spark.sparkContext.parallelize(musicas)
musics_artist = spark.createDataFrame(rdd)
musics_artist.show(10, False)

+----------------------+-------------+----------------------------+
|artista               |genero       |nome                        |
+----------------------+-------------+----------------------------+
|Charlie Brown Jr.     |Rock Nacional|Dias De Luta, Dias De Gloria|
|Charlie Brown Jr.     |Rock Nacional|Só Por Uma Noite            |
|Los Abuelos De La Nada|Rock Nacional|Mil Horas                   |
|NX Zero               |Rock Nacional|Razões E Emoções            |
|Los Abuelos De La Nada|Rock Nacional|Mil Horas                   |
|NX Zero               |Rock Nacional|Pela Última Vez             |
|Piero                 |Rock Nacional|Mi Viejo                    |
|Tihuana               |Rock Nacional|Tropa De Elite              |
|Miguel Mateos - Zas   |Rock Nacional|Cuando Seas Grande          |
|NX Zero               |Rock Nacional|Além De Mim                 |
+----------------------+-------------+----------------------------+
only showing top 10 rows



### Armazene os dados em formato parquet, particionando por artista;

In [ ]:
musics_artist.write.partitionBy('artista').parquet('artistas_generos.parquet')

### Crie um dataframe com o endpoint "Get Artist"

In [25]:
id = '1on7ZQ2pvgeQF4vmIA09x5'
get_artist = f'https://api.spotify.com/v1/artists/{id}'

r = requests.request("GET", url=get_artist, headers=headers).json()

rdd = spark.sparkContext.parallelize([r])
artist = spark.createDataFrame(rdd)
artist.show(truncate = False)

+-------------------------------------------------------------------+--------------------------------+---------------------------------------------------------------------------------------------+---------------------------------------------------------+----------------------+------------------------------------------------------------------------------------------------------------------------------------+-----------------+----------+------+-------------------------------------+
|external_urls                                                      |followers                       |genres                                                                                       |href                                                     |id                    |images                                                                                                                              |name             |popularity|type  |uri                                  |
+-----------------------------

### Crie um dataframe com o endpoint "Get Artist's Albums" e traga as músicas dos álbuns que estão no endpoint "Get Album Tracks"

In [43]:
id = '1on7ZQ2pvgeQF4vmIA09x5'
artist_albums = sp.artist_albums(id)
albums_tracks = []

for album in artist_albums['items']:
    album_name = album['name']
    album_id = album['id']
    album_tracks = sp.album_tracks(album_id)
    for track in album_tracks['items']:
        track_name = track['name']
        track_dict = {
            'album_name': album_name,
            'track_name': track_name
        }
        albums_tracks.append(track_dict)

artist_album = spark.createDataFrame(albums_tracks)
artist_album.show(10, False)

+------------------------------------------------+-------------------------------------------------------------+
|album_name                                      |track_name                                                   |
+------------------------------------------------+-------------------------------------------------------------+
|Chegou Quem Faltava - Versão do Chorão (Ao Vivo)|Cheia de Vida - Ao Vivo - Chegou Quem Faltava                |
|Chegou Quem Faltava - Versão do Chorão (Ao Vivo)|O Côro Vai Comê - Ao Vivo - Chegou Quem Faltava              |
|Chegou Quem Faltava - Versão do Chorão (Ao Vivo)|Charlie Brown Jr. - Ao Vivo - Chegou Quem Faltava            |
|Chegou Quem Faltava - Versão do Chorão (Ao Vivo)|Gimme o Anel / Sheik - Ao Vivo - Chegou Quem Faltava         |
|Chegou Quem Faltava - Versão do Chorão (Ao Vivo)|Quinta-Feira - Ao Vivo - Chegou Quem Faltava                 |
|Chegou Quem Faltava - Versão do Chorão (Ao Vivo)|Tudo Que Ela Gosta de Escutar - Ao Vivo - Cheg

### Crie um dataframe com o endpoint "Get Current User's Playlists"

In [17]:
get_user_playlist = 'https://api.spotify.com/v1/users/{}/playlists'

r = requests.request("GET", url=get_user_playlist, headers=headers).json()["items"]
playlists = []
for playlist in r:
    playlists.append({
        "name": playlist["name"],
        "tracks": playlist["tracks"]["total"]
    })

playlist = spark.createDataFrame(playlists)
playlist.show(truncate=False)

+--------------------+------+
|name                |tracks|
+--------------------+------+
|Chico               |49    |
|My Sacrifice – Creed|40    |
|Gringos             |32    |
|Playlist            |67    |
+--------------------+------+



### Crie uma tabela temporária em PySpark a partir do DataFrame de músicas dos artistas;

In [22]:
musics_artist.createOrReplaceTempView('musics_artist')

### Crie uma tabela temporária em Pyspark a partir do Dataframe de Playlists

In [23]:
playlist.createOrReplaceTempView('playlist')

### Execute uma consulta SQL que retorna os artistas do endpoint "Get Artist" que estão nas Playlists, ordenados por ordem alfabética.

In [27]:
artist.createOrReplaceTempView('artist')
# O dataframe Playlist não me retornou os artistas, por isso do campo comentado
query = '''
  select 
    name 
  from 
    artist 
  --where 
    --name in (select distinct {} from playlist) 
  order by 
    name
'''
spark.sql(query).show()

+-----------------+
|             name|
+-----------------+
|Charlie Brown Jr.|
+-----------------+



### Crie um Dataframe com o resultado da consulta e salve em parquet

In [ ]:
df = spark.sql(query)
df.write.parquet('consulta.parquet')